In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import sys
sys.path.append('./waveglow/')

import librosa
import numpy as np
import os
import glob
import json

import torch
from text import text_to_sequence, cmudict
from text.symbols import symbols
import commons
import attentions
import modules
import models
import utils

# load WaveGlow
waveglow_path = './waveglow/waveglow_256channels_universal_v5.pt' # or change to the latest version of the pretrained WaveGlow.
waveglow = torch.load(waveglow_path)['model']
waveglow = waveglow.remove_weightnorm(waveglow)
_ = waveglow.cuda().eval()

/home/nickatomlin/sources/conda/envs/py37/lib/python3.7/site-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/nickatomlin/sources/conda/envs/py37/lib/python3.7/site-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/nickatomlin/sources/conda/envs/py37/lib/python3.7/site-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv1d' has cha

In [5]:
# If you are using your own trained model
#model_dir = "./logs/your_dir/"
#hps = utils.get_hparams_from_dir(model_dir)
#checkpoint_path = utils.latest_checkpoint_path(model_dir)

# If you are using a provided pretrained model
hps = utils.get_hparams_from_file("./configs/base.json")
checkpoint_path = "pretrained/pretrained.pth"

model = models.FlowGenerator(
    len(symbols) + getattr(hps.data, "add_blank", False),
    out_channels=hps.data.n_mel_channels,
    **hps.model).to("cuda")

utils.load_checkpoint(checkpoint_path, model)
model.decoder.store_inverse() # do not calcuate jacobians for fast decoding
_ = model.eval()

cmu_dict = cmudict.CMUDict(hps.data.cmudict_path)

# normalizing & type casting
def normalize_audio(x, max_wav_value=hps.data.max_wav_value):
    return np.clip((x / np.abs(x).max()) * max_wav_value, -32768, 32767).astype("int16")

/home/nickatomlin/andrehe/glow-tts/modules.py:200: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  /opt/conda/conda-bld/pytorch_1631630742027/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:1940.)
  w_init = torch.qr(torch.FloatTensor(self.n_split, self.n_split).normal_())[0]


INFO:root:Loaded checkpoint 'pretrained/pretrained.pth' (iteration 0)


In [10]:
tst_stn = "R. S. Stovall, and G. F. Rose obtained a search warrant and examined Oswald's effects in the Paine garage." 

if getattr(hps.data, "add_blank", False):
    text_norm = text_to_sequence(tst_stn.strip(), ['english_cleaners'], cmu_dict)
    text_norm = commons.intersperse(text_norm, len(symbols))
else: # If not using "add_blank" option during training, adding spaces at the beginning and the end of utterance improves quality
    tst_stn = " " + tst_stn.strip() + " "
    text_norm = text_to_sequence(tst_stn.strip(), ['english_cleaners'], cmu_dict)
sequence = np.array(text_norm)[None, :]
print("".join([symbols[c] if c < len(symbols) else "<BNK>" for c in sequence[0]]))
x_tst = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
x_tst_lengths = torch.tensor([x_tst.shape[1]]).cuda()

@AA1@R @EH1@S stovall, @AH0@N@D @JH@IY1 @EH1@F @R@OW1@Z @AH0@B@T@EY1@N@D @EY1 @S@ER1@CH @W@AO1@R@AH0@N@T @AH0@N@D @IH0@G@Z@AE1@M@AH0@N@D @AO1@Z@W@AO0@L@D@Z @IH0@F@EH1@K@T@S @IH0@N @DH@AH0 @P@EY1@N garage.


In [20]:
x_tst, x_tst_lengths
x_tst.dtype

torch.int64

In [22]:
with torch.no_grad():
  noise_scale = .667
  length_scale = 1.0
  (y_gen_tst, *_), *_, (attn_gen, *_) = model(x_tst, x_tst_lengths, gen=True, noise_scale=noise_scale, length_scale=length_scale)
  audio = waveglow.infer(y_gen_tst, sigma=.666)
ipd.Audio(normalize_audio(audio[0].clamp(-1,1).data.cpu().float().numpy()), rate=hps.data.sampling_rate)

In [7]:
y_gen = np.load('mels.npy')

# If you are using a provided pretrained model
hps = utils.get_hparams_from_file("./configs/base.json")
checkpoint_path = "pretrained/pretrained.pth"


# normalizing & type casting
def normalize_audio(x, max_wav_value=hps.data.max_wav_value):
    return np.clip((x / np.abs(x).max()) * max_wav_value, -32768, 32767).astype("int16")

y_gen = torch.from_numpy(y_gen).cuda()

with torch.no_grad():
    audio = waveglow.infer(y_gen, sigma=.666)
ipd.Audio(normalize_audio(audio[1].clamp(-1,1).data.cpu().float().numpy()), rate=hps.data.sampling_rate)

In [11]:
ipd.Audio(normalize_audio(audio[0].clamp(-1,1).data.cpu().float().numpy()), rate=hps.data.sampling_rate)